# Signature Hashes and Validation

Perhaps the trickiest part of validating a transaction is the process of checking its signatures. A transaction typically has at least one signature per input. If there are multisig outputs being spent, there may be more. As we learned earlier, the ECDSA signature algorithm requires that for each input, we need the public key (P), the signature hash (z), and the Signature (r,s). Once these are determined, the process of verifying the signature is pretty simple.

```python
point = Point.parse(sec)
signature = Signature.parse(der)
point.verify(z, signature)  # True
```

The sec and der formats make getting the P, r and s pretty simple. The hard part is getting the actual signature hash (z). You would think that this would be easy since you can just hash the transaction. But you can't do that since the signature itself is part of the scriptSig and a signature can't sign itself.

Instead, what you need to do is to modify the transaction before actually signing it. That is, you have to compute the z in a very particular way. The procedure is as follows.

In [ ]:
# Exercise 5.1
from io import BytesIO
from tx import Tx
import helper

hex_tx = '010000000456919960ac691763688d3d3bcea9ad6ecaf875df5339e148a1fc61c6ed7a069e010000006a47304402204585bcdef85e6b1c6af5c2669d4830ff86e42dd205c0e089bc2a821657e951c002201024a10366077f87d6bce1f7100ad8cfa8a064b39d4e8fe4ea13a7b71aa8180f012102f0da57e85eec2934a82a585ea337ce2f4998b50ae699dd79f5880e253dafafb7feffffffeb8f51f4038dc17e6313cf831d4f02281c2a468bde0fafd37f1bf882729e7fd3000000006a47304402207899531a52d59a6de200179928ca900254a36b8dff8bb75f5f5d71b1cdc26125022008b422690b8461cb52c3cc30330b23d574351872b7c361e9aae3649071c1a7160121035d5c93d9ac96881f19ba1f686f15f009ded7c62efe85a872e6a19b43c15a2937feffffff567bf40595119d1bb8a3037c356efd56170b64cbcc160fb028fa10704b45d775000000006a47304402204c7c7818424c7f7911da6cddc59655a70af1cb5eaf17c69dadbfc74ffa0b662f02207599e08bc8023693ad4e9527dc42c34210f7a7d1d1ddfc8492b654a11e7620a0012102158b46fbdff65d0172b7989aec8850aa0dae49abfb84c81ae6e5b251a58ace5cfeffffffd63a5e6c16e620f86f375925b21cabaf736c779f88fd04dcad51d26690f7f345010000006a47304402200633ea0d3314bea0d95b3cd8dadb2ef79ea8331ffe1e61f762c0f6daea0fabde022029f23b3e9c30f080446150b23852028751635dcee2be669c2a1686a4b5edf304012103ffd6f4a67e94aba353a00882e563ff2722eb4cff0ad6006e86ee20dfe7520d55feffffff0251430f00000000001976a914ab0c0b2e98b1ab6dbf67d4750b0a56244948a87988ac005a6202000000001976a9143c82d7df364eb6c75be8c80df2b3eda8db57397088ac46430600'
hex_tx2 ='...'
# bytes.fromhex the tx, make stream
stream = BytesIO(bytes.fromhex(hex_txaa))
# parse the tx
t = Tx.parse(stream)
# initialize input sum

input_sum = 0
# iterate over all inputs (t.tx_ins)
for tx_in in t.tx_ins:
    # get the values from the TxIn.value method you wrote in 4.2
    value = tx_in.value()
    # add to input sum
    input_sum += value
# initialize output sum
output_sum = 0
# iterate over all outputs (t.tx_outs)
for tx_out in t.tx_outs:
    # get the amounts from the TxOut.amount property
    value = tx_out.amount
    # add to output sum
    output_sum += value
# fee is input sum - output sum
fee = input_sum - output_sum
print(fee)

#separator for formating
def separator(title):
    print()
    print('====================')
    print(title)

bitcoin_price = 6400

#satoshis in in bitcoin
whole = 100000000
# print version
version = t.version
separator('VERSION:')
print('Version number: ' + str(version))

separator('INPUTS:')
#print inputs
print(len(t.tx_ins))

input_total = 0
for tx_ins in t.tx_ins:
    print('Inputs: ' + str(tx_ins))
    print('Value: ' + str(tx_ins.value()/whole))
    input_total += tx_ins.value()
print('Input Total: ' + str(input_total/whole))

separator('OUTPUTS:')
#print outputs
output_total = 0
for tx_outs in t.tx_outs:
    print('Output: ' + str(tx_outs))
    output_total += tx_outs.amount
print('Output Total: ' + str(output_total/whole))

separator('FEE:')
#print fee
fee = (input_total - output_total)
print('Fee: ' + str(fee) + ' in satoshis or ' + str(fee/whole) + ' bitcoin' + ' or $' + str(round((fee/whole*bitcoin_price), 2)) )

separator('LOCKTIME:')
locktime = t.locktime
print('Locktime: ' + str(locktime))
    

### Test Driven Exercise

In [9]:
from io import BytesIO
from helper import (
    SIGHASH_ALL,
    int_to_little_endian,
    double_sha256
)
from tx import Tx, TxIn


class Tx(Tx):


    def sig_hash(self, input_index, hash_type):
        '''Returns the integer representation of the hash that needs to get
        signed for index input_index'''
        # create a new set of tx_ins (alt_tx_ins)
        alt_tx_ins = []
        # iterate over self.tx_ins
        for tx_in in self.tx_ins:
            # create a new TxIn that has a blank script_sig (b'') and add to alt_tx_ins
            alt_tx_ins.append(TxIn(
                prev_tx=tx_in.prev_tx,
                prev_index=tx_in.prev_index,
                script_sig=b'',
                sequence=tx_in.sequence,
            ))
        # grab the input at the input_index
        signing_input = alt_tx_ins[input_index]
        # grab the script_pubkey of the input
        script_pubkey = signing_input.script_pubkey(self.testnet)
        # the script_sig of the signing_input should be script_pubkey
        signing_input.script_sig = script_pubkey
        # create an alternate transaction with the modified tx_ins
        alt_tx = self.__class__(
            version=self.version,
            tx_ins=alt_tx_ins,
            tx_outs=self.tx_outs,
            locktime=self.locktime)
        # add the hash_type int 4 bytes, little endian
        result = alt_tx.serialize() + int_to_little_endian(hash_type, 4)
        # get the double_sha256 of the tx serialization
        s256 = double_sha256(result)
        # convert this to a big-endian integer using int.from_bytes(x, 'big')
        return int.from_bytes(s256, 'big')